In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [11]:
EPOCHS = 100

## 모델 정의

In [12]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 데이터셋 준비


In [9]:
fashion_mnist = tf.keras.datasets.fashion_mnist # 28x28 -> 10class

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습

In [13]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5182 - accuracy: 0.8108 - val_loss: 0.4494 - val_accuracy: 0.8385
Epoch 2/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3800 - accuracy: 0.8597 - val_loss: 0.3914 - val_accuracy: 0.8576
Epoch 3/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3479 - accuracy: 0.8722 - val_loss: 0.4159 - val_accuracy: 0.8470
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3285 - accuracy: 0.8789 - val_loss: 0.3651 - val_accuracy: 0.8678
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3130 - accuracy: 0.8845 - val_loss: 0.3521 - val_accuracy: 0.8750
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3009 - accuracy: 0.8879 - val_loss: 0.3657 - val_accuracy: 0.8742
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2877 - accuracy: 0.8924 - val_loss: 0.3427 - val_ac

1875/1875 [==============================] - 5s 2ms/step - loss: 0.1372 - accuracy: 0.9473 - val_loss: 0.6143 - val_accuracy: 0.8804
Epoch 58/100
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1366 - accuracy: 0.9470 - val_loss: 0.6138 - val_accuracy: 0.8760
Epoch 59/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1400 - accuracy: 0.9467 - val_loss: 0.5828 - val_accuracy: 0.8782
Epoch 60/100
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1360 - accuracy: 0.9480 - val_loss: 0.5714 - val_accuracy: 0.8830
Epoch 61/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1433 - accuracy: 0.9469 - val_loss: 0.5591 - val_accuracy: 0.8811
Epoch 62/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1318 - accuracy: 0.9498 - val_loss: 0.6469 - val_accuracy: 0.8747
Epoch 63/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1341 - accuracy: 0.9491 - val_loss: 0.6058 - val_accuracy

## 모델 변경(DenseNet 사용)

In [23]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32, use_bias=False)
        
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64, use_bias=False)
        
        self.batch4 = tf.keras.layers.BatchNormalization()
        self.dense4 = tf.keras.layers.Dense(128, use_bias=False)
        
        
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)
        
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x,h], axis=-1)
        
        
        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x,h], axis=-1)
        
        
        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x,h], axis=-1)
        
        
        return self.dense5(x)

In [24]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5150 - accuracy: 0.8157 - val_loss: 0.4332 - val_accuracy: 0.8422
Epoch 2/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3992 - accuracy: 0.8545 - val_loss: 0.3994 - val_accuracy: 0.8547
Epoch 3/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3663 - accuracy: 0.8647 - val_loss: 0.4285 - val_accuracy: 0.8442
Epoch 4/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3451 - accuracy: 0.8733 - val_loss: 0.4023 - val_accuracy: 0.8555
Epoch 5/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3269 - accuracy: 0.8797 - val_loss: 0.3747 - val_accuracy: 0.8667
Epoch 6/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3169 - accuracy: 0.8842 - val_loss: 0.3659 - val_accuracy: 0.8722
Epoch 7/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3062 - accuracy: 0.8860 - val_loss: 0.3981 - val_ac

Epoch 57/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1811 - accuracy: 0.9296 - val_loss: 0.3835 - val_accuracy: 0.8829
Epoch 58/100
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1815 - accuracy: 0.9310 - val_loss: 0.3883 - val_accuracy: 0.8800
Epoch 59/100
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1810 - accuracy: 0.9309 - val_loss: 0.3917 - val_accuracy: 0.8805
Epoch 60/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1825 - accuracy: 0.9305 - val_loss: 0.3845 - val_accuracy: 0.8849
Epoch 61/100
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1805 - accuracy: 0.9317 - val_loss: 0.3998 - val_accuracy: 0.8789
Epoch 62/100
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1794 - accuracy: 0.9315 - val_loss: 0.3898 - val_accuracy: 0.8820
Epoch 63/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1790 - accuracy: 0.9312 - val_loss: 0